In [34]:
from urllib import request
import time

url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRgolORTuMr7IGUnxT3j80TAvNA86K_PoFrGgaciICuJBwXGXa2'

tries = 4
delay = 3
backoff = 1
mtries, mdelay = tries, delay

while mtries > 0:
    try:
        print(mtries)
        response = request.urlopen(url)
        print('Success. Exiting..')
        break
    except (ValueError, request.HTTPError, request.URLError) as ex:
        print('Exception. Retrying..')
#         msg = "%s, Retrying in %d seconds..." % (str(ex), mdelay)
        time.sleep(mdelay)
        mtries -= 1
        mdelay *= backoff
print('END')

4
Success. Exiting..
END


In [35]:
tries = 4
delay = 3
backoff = 1
mtries, mdelay = tries, delay

while mtries > 0:
    try:
        print(mtries)
        url = 'INVALID URL'
        response = request.urlopen(url)
        print('Success. Exiting..')
        break
    except (ValueError, request.HTTPError, request.URLError) as ex:
        print('Exception. Retrying..')
#         msg = "%s, Retrying in %d seconds..." % (str(ex), mdelay)
        time.sleep(mdelay)
        mtries -= 1
        mdelay *= backoff
print('END')

4
Exception. Retrying..
3
Exception. Retrying..
2
Exception. Retrying..
1
Exception. Retrying..
END


In [37]:
def download_url_with_retries(url):
    tries = 4
    delay = 3
    backoff = 1
    mtries, mdelay = tries, delay

    while mtries > 0:
        try:
            print(mtries)
            response = request.urlopen(url)
            print('Success. Exiting..')
            break
        except (ValueError, request.HTTPError, request.URLError) as ex:
            print('Exception Type: ' + type(ex).__name__ +' .. Retrying..')
    #         msg = "%s, Retrying in %d seconds..." % (str(ex), mdelay)
            time.sleep(mdelay)
            mtries -= 1
            mdelay *= backoff
    print('END')

In [38]:
url = 'INVALID URL'
download_url_with_retries(url)

4
Exception Type: ValueError .. Retrying..
3
Exception Type: ValueError .. Retrying..
2
Exception Type: ValueError .. Retrying..
1
Exception Type: ValueError .. Retrying..
END


In [39]:
url = 'http://www.INVALIDURLasdasdsadasdasd.com'
download_url_with_retries(url)

4
Exception Type: URLError .. Retrying..
3
Exception Type: URLError .. Retrying..
2
Exception Type: URLError .. Retrying..
1
Exception Type: URLError .. Retrying..
END


In [5]:
import time
from functools import wraps


def retry(ExceptionToCheck, tries=4, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck as e:
                    msg = "%s, Retrying in %d seconds..." % (str(e), mdelay)
                    if logger:
                        logger.warning(msg)
                    else:
                        print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry

In [6]:
@retry(Exception, tries=4)
def test_fail(text):
    raise Exception("Fail")

test_fail("it works!")

Fail, Retrying in 3 seconds...
Fail, Retrying in 6 seconds...
Fail, Retrying in 12 seconds...


Exception: Fail

In [8]:
@retry(Exception, tries=4)
def test_success(text):
    print("Success: ", text)

test_success("it works!")

Success:  it works!


In [12]:
import random

@retry(Exception, tries=4)
def test_random(text):
    x = random.random()
    if x < 0.5:
        raise Exception("Fail")
    else:
        print("Success: ", text)

test_random("it works!")

Fail, Retrying in 3 seconds...
Success:  it works!


In [13]:
import random

@retry((NameError, IOError), tries=20, delay=1, backoff=1)
def test_multiple_exceptions():
    x = random.random()
    if x < 0.40:
        raise NameError("NameError")
    elif x < 0.80:
        raise IOError("IOError")
    else:
        raise KeyError("KeyError")

In [14]:
from urllib import request

In [18]:
@retry((request.URLError, request.HTTPError), tries=4, delay=3, backoff=2)
def urlopen_with_retry(success):
    
    x = random.random()
    if x < 0.40:
        raise NameError("NameError")
    elif x < 0.80:
        raise IOError("IOError")
    else:
        raise KeyError("KeyError")
        return request.urlopen(url)

In [20]:
url = 'http://www.example.com'
urlopen_with_retry(url)

In [26]:
url='http://vis.iaai.com/resizer?imagekeys=25607745~I1&Width=640&Height=480'

for i in range(0,100):
    try:
        response = request.urlopen(url)
    except Exception as ex:
        print(i)
        raise ex

In [27]:
request.URLError
request.HTTPError

urllib.error.HTTPError

In [34]:
import time
from functools import wraps


def retry(ExceptionToCheck, tries=4, delay=3, backoff=1):
    def deco_retry(f):
        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck as e:
                    msg = "%s, Retrying in %d seconds..." % (str(e), mdelay)
                    print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry

In [33]:
# ALWAYS FAILS
@retry(Exception, tries=4)
def test_fail(text):
    raise Exception("Fail")

test_fail("it works!")

Fail, Retrying in 3 seconds...
Fail, Retrying in 6 seconds...
Fail, Retrying in 12 seconds...


Exception: Fail

In [26]:
from urllib import request
import time

tries = 4
delay = 3
backoff = 1
mtries, mdelay = tries, delay

while mtries > 0:
    try:
        print(mtries)
        response = request.urlopen('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRgolORTuMr7IGUnxT3j80TAvNA86K_PoFrGgaciICuJBwXGXa2')
        print('Success. Exiting..')
        break
    except (request.HTTPError, request.URLError) as ex:
        print('Exception. Retrying..')
#         msg = "%s, Retrying in %d seconds..." % (str(ex), mdelay)
        time.sleep(mdelay)
        mtries -= 1
        mdelay *= backoff
print('END')

4
Success. Exiting..
END
